Building a Chatbot with history

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GOOGLE_API_KEY"]=os.getenv("GEMINI_KEY")
#Langchain Tracing
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_KEY")

In [3]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000019917A60050>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019917A60D70>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
response=llm.invoke([HumanMessage(content="Hii, My name is Sachin Goyal.I am currently pursuing my B.Tech(3rd year) from Thapar University in CSE,i am currently practising Generative Ai")])
response.content

"Hi Sachin,\n\nIt's great to meet you! It's impressive that you're already practicing Generative AI in your third year of B.Tech at Thapar University. \n\nThat's a cutting-edge field with a lot of potential.\n\nWhat specific areas of Generative AI are you focusing on? Are you working on any particular projects? I'd love to hear more about your work!\n\nI can also offer some information or resources if you have any questions about Generative AI.\n"

In [5]:
from langchain_core.messages import AIMessage
llm.invoke(
    [
        HumanMessage(content="Hii, My name is Sachin Goyal.I am currently pursuing my B.Tech(3rd year) from Thapar University in CSE,i am currently practising Generative Ai.what will you suggest me for a better learning?") ,
        AIMessage(content="Hi Sachin,\n\nIt's great to meet you!  It's impressive that you're already practicing Generative AI during your B.Tech in CSE at Thapar University.  That's a very forward-looking skillset to be developing.  What specific areas of Generative AI are you focusing on right now?  I'd be interested to hear more about your projects.") ,
        HumanMessage(content="What is my name?")
    ]
)


AIMessage(content='Your name is Sachin Goyal. 😊  \n\nI remember that from our first interaction!  Is there anything else I can help you with today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 152, 'total_tokens': 185, 'completion_time': 0.06, 'prompt_time': 0.006521219, 'queue_time': 0.243865931, 'total_time': 0.066521219}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--d40138d8-c4a8-4982-a9c8-ebc1feb4c0f1-0', usage_metadata={'input_tokens': 152, 'output_tokens': 33, 'total_tokens': 185})

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={} #storing all the chat with their id to uniquely define and prevent chat mixing
def get_session_history(session_id)->BaseChatMessageHistory:
    if session_id not in store:  #if new chat then store the chat going on
        store[session_id]=ChatMessageHistory()
    return store[session_id]
with_message_history=RunnableWithMessageHistory(llm,get_session_history)

In [7]:
#declaring session id chat1 for this chat for a user
config={"configurable":{"session_id":"chat1"}}

response=with_message_history.invoke (
    [ HumanMessage(content="Hii, My name is Sachin Goyal.I am currently pursuing my B.Tech(3rd year) from Thapar University in CSE,i am currently practising Generative Ai.what will you suggest me for a better learning?") ]
    ,config=config )
response

AIMessage(content="Hey Sachin, it's great to hear you're diving into the exciting world of Generative AI! As a large language model myself, I'm happy to offer some suggestions for your learning journey. \n\nSince you're already in your 3rd year of a B.Tech in CSE at Thapar University, you have a strong foundation. Here's a structured approach to enhance your Generative AI skills:\n\n**1. Solidify Your Fundamentals:**\n\n* **Deep Learning:** Ensure you have a solid grasp of deep learning concepts:\n    * **Neural Networks:** Architectures (CNNs, RNNs, Transformers), activation functions, backpropagation, optimization algorithms.\n    * **Machine Learning:** Supervised, unsupervised, and reinforcement learning.\n    * **Python & Libraries:**  Master Python and essential libraries like TensorFlow, PyTorch, Keras.\n\n* **Mathematics:**  Brush up on linear algebra, calculus, and probability/statistics – they are crucial for understanding deep learning algorithms.\n\n**2. Dive into Generativ

In [8]:
#see this time it says i don't know about you name
#basically the session id is a new one so no previous chat available it will be stored for the first time

config={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="what is my name")],config=config
)
response

AIMessage(content="As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 13, 'total_tokens': 55, 'completion_time': 0.076363636, 'prompt_time': 0.00184683, 'queue_time': 0.244350179, 'total_time': 0.078210466}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--b36d84c7-0493-41f2-9623-49b650b03136-0', usage_metadata={'input_tokens': 13, 'output_tokens': 42, 'total_tokens': 55})

In [9]:
config={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    [HumanMessage(content="ok tell me what is my name")],config=config
)
response
#see it remembers my name

AIMessage(content="Your name is Sachin Goyal.  😊\n\nIt's nice to meet you, Sachin! \n\nDo you have any other questions for me? Perhaps about Generative AI?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 776, 'total_tokens': 816, 'completion_time': 0.072727273, 'prompt_time': 0.032538779, 'queue_time': 0.24680955000000002, 'total_time': 0.105266052}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--e34cd2a3-0aa6-4671-8992-e68956bc53eb-0', usage_metadata={'input_tokens': 776, 'output_tokens': 40, 'total_tokens': 816})

In [10]:
config={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke([HumanMessage(content="My name is John")],config=config)
response

AIMessage(content="Hi John! It's nice to meet you. What can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 67, 'total_tokens': 92, 'completion_time': 0.045454545, 'prompt_time': 0.0035126, 'queue_time': 0.24696273900000001, 'total_time': 0.048967145}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--4df9bfa1-7df4-4a5c-a535-838f56da5d41-0', usage_metadata={'input_tokens': 67, 'output_tokens': 25, 'total_tokens': 92})

In [11]:
config={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="what is my name")],config=config
)
response
#this time remembers the name

AIMessage(content='Your name is John.  I remember! 😊  Anything else I can help you with?  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 103, 'total_tokens': 126, 'completion_time': 0.041818182, 'prompt_time': 0.00491438, 'queue_time': 0.2457163, 'total_time': 0.046732562}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--50d7ee90-e7bd-4cf1-933f-6ef16cae1d6a-0', usage_metadata={'input_tokens': 103, 'output_tokens': 23, 'total_tokens': 126})

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.


HOW IT WORKS

-->we create a prompt where system command is there to llm

-->we create a variable of MessagePlaceholder which will place all the previous msg+new one

-->we create a chain of prompt and llm

-->for chat history we have Runnable with msgHistory which gets session id and
    retrieve all the prev msg history and provide it to chain

-->the llm will generate the response based on the prev history

In [29]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
     [
         ("system","You are a helpful assistant ,communicate with the user according"),
         MessagesPlaceholder(variable_name="messages")
     ]
 )
chain=prompt|llm

In [30]:
chain.invoke({"messages":[HumanMessage(content="Hii ,My name is Sachin Goyal")]})

AIMessage(content="Hi Sachin Goyal, \n\nIt's nice to meet you! 👋\n\nHow can I help you today? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 28, 'total_tokens': 57, 'completion_time': 0.052727273, 'prompt_time': 0.00206893, 'queue_time': 0.24470646000000001, 'total_time': 0.054796203}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--cca8e2e3-dca9-4474-9467-8309fb70f8ce-0', usage_metadata={'input_tokens': 28, 'output_tokens': 29, 'total_tokens': 57})

In [31]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [32]:
config={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    {'message':[HumanMessage(content="My name is Sachin Goyal")]},config=config
)
response

AIMessage(content="Hi Sachin Goyal! \n\nIt's nice to officially meet you.  \n\nHow can I help you explore the world of Generative AI today? \n\n\n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 1063, 'total_tokens': 1100, 'completion_time': 0.067272727, 'prompt_time': 0.036000428, 'queue_time': 0.249809101, 'total_time': 0.103273155}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--6b372dec-a552-45b1-a9d3-2a455691d089-0', usage_metadata={'input_tokens': 1063, 'output_tokens': 37, 'total_tokens': 1100})

In [35]:
config={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    {'message':[HumanMessage(content="can you tell me what is my name,age and my fav hobby")]},config=config
)
response

AIMessage(content="As an AI, I don't have access to personal information about you like your age or favorite hobby.  \n\nI do know your name is Sachin Goyal, though! \n\nWould you like to tell me about your favorite hobby? I'm always eager to learn new things. 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 1244, 'total_tokens': 1308, 'completion_time': 0.116363636, 'prompt_time': 0.048218107, 'queue_time': 0.256127273, 'total_time': 0.164581743}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--1eedd660-0108-43fd-b3fb-7887b19644ec-0', usage_metadata={'input_tokens': 1244, 'output_tokens': 64, 'total_tokens': 1308})

What if we have multiple inputs to be taken in prompt
--> then there is a change in with_message_history
    we need to define a correct key to use to save chat to history

In [17]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
     [
         ("system","You are a helpful assistant ,communicate with the user according in {language}"),
         MessagesPlaceholder(variable_name="messages")
     ]
 )
chain=prompt|llm

In [18]:
chain.invoke({"messages":[HumanMessage(content="Hii ,My name is Sachin Goyal")],"language":"Hindi"})

AIMessage(content='नमस्ते सचिन गोयल जी! 👋 \n\nमैं आपकी मदद करने के लिए यहाँ हूँ।  आप क्या पूछना चाहेंगे? \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 30, 'total_tokens': 73, 'completion_time': 0.078181818, 'prompt_time': 0.002842749, 'queue_time': 0.265423542, 'total_time': 0.081024567}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c263061b-4f75-4564-99bf-f7724e639075-0', usage_metadata={'input_tokens': 30, 'output_tokens': 43, 'total_tokens': 73})

In [19]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

whenever we reuse the older session for ex chat1 then it loads the old history and as we used english by default so now declaring any other language may not work for that and will print in english

In [20]:
config={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="can you tell me my name ")],"language":"Hindi"},config=config
)
response

AIMessage(content='You said your name is Sachin Goyal.  \n\nIs there anything else I can help you with regarding Generative AI? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 1022, 'total_tokens': 1051, 'completion_time': 0.052727273, 'prompt_time': 0.083615019, 'queue_time': 0.24546717099999998, 'total_time': 0.136342292}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--4c978b49-57fb-4ee7-bb9e-0407c4277540-0', usage_metadata={'input_tokens': 1022, 'output_tokens': 29, 'total_tokens': 1051})

In [21]:
config={"configurable":{"session_id":"chat5"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="My name is Sachin Goyal")],"language":"Hindi"},config=config
)
response

AIMessage(content='नमस्ते Sachin Goyal! \n\nमुझे आपसे बात करने में खुशी हो रही है। \n\nआप क्या पूछना चाहते हैं? मैं आपकी मदद करने के लिए तैयार हूँ।  😊\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 28, 'total_tokens': 78, 'completion_time': 0.090909091, 'prompt_time': 0.00207719, 'queue_time': 0.2438898, 'total_time': 0.092986281}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--660c16cf-c1e5-47f5-bf88-54607de5c935-0', usage_metadata={'input_tokens': 28, 'output_tokens': 50, 'total_tokens': 78})

In [22]:
config={"configurable":{"session_id":"chat5"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="can you tell me my name ")],"language":"Hindi"},config=config
)
response

AIMessage(content='बिल्कुल! आपका नाम **Sachin Goyal** है। \n\nक्या कुछ और बताना चाहेंगे? 😉\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 93, 'total_tokens': 123, 'completion_time': 0.054545455, 'prompt_time': 0.004160551, 'queue_time': 0.247050908, 'total_time': 0.058706006}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--148aa70f-09c9-4325-8ef9-bd7ab47f8e1c-0', usage_metadata={'input_tokens': 93, 'output_tokens': 30, 'total_tokens': 123})

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [41]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy='last', # keep the msgs from last with tokens<=max_tokens
    token_counter=llm,
    allow_partial=False, #If a msg pushes total tokens over 45, it won’t partially include it instead, it will stop adding messages.
    include_system=True, # as system guides the model what should it do so it must be present anyhow even after trimming msgs
    start_on='human'  #look for the most recent HumanMessage and start including from there.
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

maybe the trimmer trimmed the ice cream part here so not working check next code section

In [40]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain=(
        RunnablePassthrough.assign(messages=itemgetter("messages") |trimmer)
       | prompt
       | llm
)
response=chain.invoke(
    {"messages":messages+[HumanMessage(content="what ice cream do i like")],"language":"Hindi"}
)
response

AIMessage(content="As a helpful assistant, I don't have access to your personal preferences like your favorite ice cream flavor. \n\nWhat's your favorite ice cream?  🍦😋\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 79, 'total_tokens': 117, 'completion_time': 0.069090909, 'prompt_time': 0.00382009, 'queue_time': 0.24639867999999998, 'total_time': 0.072910999}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--f7d749dd-0e1c-48bb-9848-c9177349a531-0', usage_metadata={'input_tokens': 79, 'output_tokens': 38, 'total_tokens': 117})

In [38]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain=(
        RunnablePassthrough.assign(messages=itemgetter("messages") |trimmer)
       | prompt
       | llm
)
response=chain.invoke(
    {"messages":messages+[HumanMessage(content="what was my question regarding maths")],"language":"Hindi"}
)
response

AIMessage(content='You asked "what\'s 2 + 2?" 😊  \n\n\n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 79, 'total_tokens': 107, 'completion_time': 0.050909091, 'prompt_time': 0.00374958, 'queue_time': 0.24602295, 'total_time': 0.054658671}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--40840605-7a2e-4e23-9ea6-2317a375bf97-0', usage_metadata={'input_tokens': 79, 'output_tokens': 28, 'total_tokens': 107})

Wrapping with Message history (summary only)


In [26]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

In [27]:
config={"configurable":{"session_id":"chat6"}}
response=with_message_history.invoke(
    {"messages":messages+[HumanMessage(content="what is my name")],"language":"Hindi"},config=config
)
response

AIMessage(content='मैं आपको कुछ भी नहीं जानता! \n\nमैं एक बड़ा भाषा मॉडल हूँ, इसलिए मुझे याद नहीं रहता कि हमने पहले क्या बात की थी या आपका नाम क्या है।  \n\nक्या आप मुझे बताना चाहेंगे? 😊\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 79, 'total_tokens': 139, 'completion_time': 0.109090909, 'prompt_time': 0.00392919, 'queue_time': 0.243353689, 'total_time': 0.113020099}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--488684ee-88d2-4ebe-8067-e1e13060ce26-0', usage_metadata={'input_tokens': 79, 'output_tokens': 60, 'total_tokens': 139})

In [28]:
config={"configurable":{"session_id":"chat6"}}
response=with_message_history.invoke(
    {"messages":messages+[HumanMessage(content="My name is Sachin,can you tell me the meaning of this ")],"language":"Hindi"},config=config
)
response

AIMessage(content='Sachin नाम का अर्थ "सफलता" या "विजय" होता है। यह एक बहुत ही सुन्दर और प्रसिद्ध नाम है। \n\nक्या आप कोई और जानकारी चाहते हैं? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 72, 'total_tokens': 118, 'completion_time': 0.083636364, 'prompt_time': 0.00376628, 'queue_time': 0.24513489, 'total_time': 0.087402644}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--86410241-3ebb-413b-a363-d40907569a0a-0', usage_metadata={'input_tokens': 72, 'output_tokens': 46, 'total_tokens': 118})